In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy.stats import pearsonr
random.seed(1234)
np.random.seed(1234)
np.random.rand()

0.1915194503788923

In [32]:
class Buyer:
    # This class defines a buyer. A buyer has a reservation value above
    # which he will not trade. Since each buyer can only trade once, we
    # record whether a buyer traded or not. A buyer generates a bid price
    # (which varies depending if the simulation runs with or without
    # constraint)
    
    def __init__(self, quantity):
        self.Value = []
        self.Traded = 0
        self.Quantity = quantity
    
    # Generate a bid offer
    def formBidPrice(self):
        potentialBid = self.Value[self.Traded] - (np.random.rand() * (self.Value[self.Traded] - 0))
            
        return potentialBid

In [33]:
class Seller:
    # This class defines a seller. A seller has a reservation cost below
    # which he will not trade. Since each seller can only trade once, we
    # record whether a seller traded or not. A seller generates a bid price
    # (which varies depending if the simulation runs with or without
    # constraint)
    
    def __init__(self, quantity):
        self.Cost = []
        self.Traded = 0
        self.Quantity = quantity
    
    # Generate ask offer
    def formAskPrice(self):
        potentialAsk = self.Cost[self.Traded] + ((np.random.rand() * np.absolute(300 - self.Cost[self.Traded])))

        return potentialAsk

In [34]:
def initializeBook(maxCost):
    # Initialize order book variables. Record the best standing bid and ask,
    # the ID of the best bidders (index number in buyer/seller vectors), 
    # transaction price (initialized to 0) and logical variables to determine
    # if there currently is a standing bid or ask. Initially there are no best 
    # bidders, so index of each is 0. The order book vector also records the
    # added value of a given trade (initialized to 0)

    # First index is the best bid (initialized to 0)
    # Second index is the best bid ID (initialized to 0, nobody)
    # Third index is the logical variable for current bid (1 for true, initialized to 0)
    # Fourth index is the best ask (initialized to the max cost for sellers)
    # Fifth index is the best ask ID (initialized to 0, nobody)
    # Sixth index is the logical variable for current ask (1 for true, initialized to 0).
    # Seventh index is the transaction price (initialized to 0)  
    
    orderBookValues = [0 ,0 , 0, maxCost + 1,0, 0, 0, 0] 
    return orderBookValues

In [35]:
def doTrade(buyers, sellers, bookValues, numberTraders, Quantity):
    # This function operates the double auction order book. The function
    # determines with 50% probability whether the next trader to submit an
    # offer is a buyer or a seller. Then, it compares the new bid/ask and
    # executes a trade if it satisfies the counterposing best standing offer,
    # or it takes the place as best offer of its kind if it is better than the
    # currently standing one (for example, if a new bid is submitted and it is
    # higher than the best ask, a trade will occur, but if it's not higher than
    # the best ask but higher than the best standing bid it will take its
    # place. A similar process takes place if a seller submits an ask). The
    # function returns an updated state of the order book to update simulation.
    
    # Initialize return vector to current book values
    updatedValues = bookValues
    

    # Randomly choose between a buyer and a seller (belor 0.5 we select a
    # buyer, otherwise we select a seller).
    traderDeterminer = np.random.rand()
    # Process as buyer
    if traderDeterminer <0.5:
        
        # Initialize a new buyer ID (at first 0)
        newBuyer = -1
    
        #Choose a random buyer that has yet not traded. Lock selection of a
        # buyer in a while loop until it chooses a buyer that hasn't traded
        while newBuyer == -1:
            # Choose a random index in the buyers vector to select a buyer
            randomIndex = np.random.randint(0, numberTraders)
            if buyers[randomIndex].Traded != Quantity:
                newBuyer = randomIndex
        
        
        # Make buyer generate a random bid based on its reservation price and
        # simulation constraint
        newBid = buyers[newBuyer].formBidPrice()
        
        # Do a trade if there is currently a standing ask that the bid can
        # trade with. Check the logical variable and the value of the best
        # standing ask
        if (updatedValues[5] == 1) & (newBid > updatedValues[3]):
            # Set the transaction price to the best ask value
            updatedValues[6] = updatedValues[3]
            # Record surplus added by the trade
            # updatedValues[7] = buyers[newBuyer].Value - sellers[updatedValues[4]].Cost
            # Update ID of buyer
            updatedValues[1] = newBuyer

        
        # If the there is no trade, set the bid as best bid if it is higher
        # than the currently standing best bid, even if it doesn't satisfy the
        # ask or if there currently is no ask
        else:
            if newBid > updatedValues[0]:
                # Set new bid as best bid, and update the ID of bidder
                updatedValues[0]= newBid
                updatedValues[1] = newBuyer
                # Set logical variable for standing bid to true
                updatedValues[2] = 1
                # If new bid is lower than best bid, do nothing
    # Process a seller
    else:
        # Initialize a new seller ID (at first 0)
        newSeller = -1
        # Choose a random seller that has yet not traded. Lock selection of a
        # seller in a while loop until it chooses a seller that hasn't traded
        while newSeller == -1:
            # Choose a random index in the sellers vector to select a seller
            randomIndex = np.random.randint(0, numberTraders)
            if sellers[randomIndex].Traded != Quantity:
                newSeller = randomIndex
        
        # Make seller generate a random ask based on its reservation cost and
        # simulation constraint
        newAsk = sellers[newSeller].formAskPrice()
        
        # Do a trade if there is currently a standing bid that the ask can
        # trade with. Check the logical variable and the value of the best
        # standing bid
        if (updatedValues[2] == 1) & (updatedValues[0] > newAsk):
            # Set the transaction price to the best bid value
            updatedValues[6] = updatedValues[0]
            # Record surplus added by the trade
            #updatedValues[7] = buyers[updatedValues[1]].Value - sellers[newSeller].Cost
            # Record ID of seller
            updatedValues[4] = newSeller
        # If the there is no trade, set the ask as best ask if it is lower
        # than the currently standing best ask, even if it doesn't satisfy the
        # bid or if there currently is no bid
        else:
            if newAsk < updatedValues[3]:
                # Set new ask as best ask, and update the ID of bidder
                updatedValues[3] = newAsk
                updatedValues[4] = newSeller
                # Set logical variable for standing ask to true
                updatedValues[5] = 1
                # If new ask is higher than best ask, do nothing
       
    return updatedValues

In [36]:
def Average(lst): 
    return sum(lst)/len(lst)

def Rank(vector):
    a={}
    rank=1
    for num in sorted(vector):
        if num not in a:
            a[num]=rank
            rank=rank+1
    return[a[i] for i in vector]

In [37]:
# Set pre-determined parameter
numberTraders = 3
Quantity = 3
maxCost = 110

# Run simulation with 200 iteration
iterations = 2000

# Create vector holding all the values and costs
Values = [[150, 135, 120], [147, 132, 117], [144, 129, 114]]
Costs = [[50, 75, 100], [55, 80, 105], [60, 85, 110]]

# Create vector holding autocorreltion of price change
# and correlation of transaction order and costs/values
auto_corr = []
trx_order_seller = []
trx_order_buyer = []
q = []
p = []
hist = []

# Create vector holding all buyers
buyers = []  
for i in range(numberTraders):
    buyers.append(Buyer(Quantity))

# Create vector holding all sellers
sellers = []
for i in range(numberTraders):
    sellers.append(Seller(Quantity))

print("Please wait......")
for i in range(10000):
    # Initialize all buyers to not traded state and give them a reservation
    # price (random variable bounded above)
    
    valueVec = np.asarray(Values)

    for i in range(numberTraders):
        valueVec[i].sort()
        valueVec[i] = valueVec[i][::-1]
        buyers[i].Value = valueVec[i]
        buyers[i].Traded = 0
    
    # Initialize all sellers to not traded state and give them a reservation
    # cost (random variable bounded below)
    
    costVec = np.asarray(Costs)

    for i in range(numberTraders):
        costVec[i].sort()
        sellers[i].Cost = costVec[i]
        sellers[i].Traded = 0


    # Initialize the order book vector. Please see function description for the
    # values held in each index   
    orderBookValues = initializeBook(maxCost)

    # Initialize vector with transaction prices (update as iterations execute).
    # Let the length of the vector be the maximum number of iterations, then
    # discard leftover indexes initialized to zero for efficiency
    transactionPrices = []

    # Initial quantity traded is 0
    #quantity = 0

    tradedValues = []
    tradedCosts  = []
    h = []

    for i in range(iterations):
        # Stop the loop if all buyers and sellers have already traded all their goods.
        count = 0
        for i in range(numberTraders):
            if buyers[i].Traded == Quantity:
                count += 1
        if count == numberTraders:
            break
        # Attempt a trade or a new bid/ask (report update if trade occurs). 
        orderBookValues = doTrade(buyers, sellers, orderBookValues, numberTraders, Quantity)
    
        # Record transaction price, update surplus and quantity, mark traders 
        # to record that they have traded, and reinitialize the order book
        # if a trade occured
        if orderBookValues[6] > 0:
            transactionPrices.append(orderBookValues[6])
            #surplus = surplus + orderBookValues[7]
            tradedValues.append(buyers[orderBookValues[1]].Value[buyers[orderBookValues[1]].Traded])
            tradedCosts.append(sellers[orderBookValues[4]].Cost[sellers[orderBookValues[4]].Traded])
            h.append([sellers[orderBookValues[4]].Cost[sellers[orderBookValues[4]].Traded], 
                      buyers[orderBookValues[1]].Value[buyers[orderBookValues[1]].Traded], 
                      orderBookValues[6]])
            buyers[orderBookValues[1]].Traded += 1
            sellers[orderBookValues[4]].Traded += 1
            orderBookValues = initializeBook(maxCost)
            
        
    # gen transaction order and price change
    order = []
    for i in range(1, len(transactionPrices) + 1):
        order.append(i)

    all_priceChange = []
    for i in range(1, len(transactionPrices)):
        all_priceChange.append(transactionPrices[i] - transactionPrices[i - 1])

    priceChange = []
    lag_priceChange = []

    for i in range(1, len(all_priceChange)):
        priceChange.append(all_priceChange[i])
        lag_priceChange.append(all_priceChange[i-1])

    # Calculate correlation
    price_auto_corr, _ = pearsonr(priceChange, lag_priceChange)
    seller_corr, _ = pearsonr(Rank(tradedCosts), order)
    buyer_corr, _ = pearsonr(Rank(tradedValues), order)

    auto_corr.append(price_auto_corr)
    trx_order_seller.append(seller_corr)
    trx_order_buyer.append(buyer_corr)
    q.append(len(order))
    hist.append(h)

print("The simulation is done")

print("average auto corr =", Average(auto_corr))
print("average trx seller =", Average(trx_order_seller))
print("average trx buyer =", Average(trx_order_buyer))
print("average q =", Average(q))



save = -1
while (save != 0) & (save != 1):
    save = int(input('Please enter ''1'' to save , ''0'' otherwise:  '))

if save == 1:
    dict = {
        "auto_corr": auto_corr,
        "trx_order_seller": trx_order_seller,
        "trx_order_buyer": trx_order_buyer
    }
    df = pd.DataFrame(dict)
    df.to_stata("simulation_output_small.dta")
    


Please wait......
The simulation is done
average auto corr = -0.456538759243737
average trx seller = 0.8089583333333284
average trx buyer = -0.7827899999999932
average q = 9.0
Please enter 1 to save , 0 otherwise:  1


In [10]:
times = []  # will record number of transactions for every experiment
header = ['cost', 'value', 'price']

for i in hist:
    df = pd.DataFrame(i, columns=header)
    df['seller'] = df['cost'].rank()  # calculate seller rank by costs
    df['buyer'] = df['value'].rank()  # calculate buyer rank by values
    df['time'] = df.index+1  # transaction order is index+1
    times.append(len(df))

In [14]:
# An example of each experiment
df

,cost,value,price,seller,buyer,time
0,50,147,93.010995,1.0,8.0,1
1,55,132,124.594678,2.0,5.0,2
2,60,150,88.725003,3.0,9.0,3
3,80,144,125.252442,5.0,7.0,4
4,85,135,122.451975,6.0,6.0,5
5,75,120,117.175656,4.0,3.0,6
6,100,129,123.270485,7.0,4.0,7
7,105,117,115.666596,8.0,2.0,8
8,110,114,112.840404,9.0,1.0,9
